https://github.com/pysal/mgwr/pull/60

In [1]:
import sys
sys.path.append("C:/Users/msachde1/Downloads/Research/Development/mgwr")

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

from mgwr.gwr import GWR
from spglm.family import Gaussian, Binomial, Poisson
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
import multiprocessing as mp
pool = mp.Pool()
from scipy import linalg
import numpy.linalg as la
from scipy import sparse as sp
from scipy.sparse import linalg as spla
from spreg.utils import spdot, spmultiply
from scipy import special
import libpysal as ps
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import copy
from collections import namedtuple
import spglm

#### Clearwater data - downloaded from link: https://sgsup.asu.edu/sparc/multiscale-gwr

In [3]:
data_p = pd.read_csv("C:/Users/msachde1/Downloads/logistic_mgwr_data/landslides.csv") 

In [4]:
data_p.head()

,UserID,X,Y,Elev,Slope,SinAspct,CosAspct,AbsSouth,Landslid,DistStrm
0,1,616168.5625,5201076.5,1450.475,27.44172,0.409126,-0.912478,24.1499,1,8.506
1,2,624923.8125,5201008.5,1567.476,21.88343,-0.919245,-0.393685,66.8160,1,15.561
2,3,615672.0000,5199187.5,1515.065,38.81030,-0.535024,-0.844837,32.3455,1,41.238
3,4,615209.3125,5199112.0,1459.827,26.71631,-0.828548,-0.559918,55.9499,1,17.539
4,5,616354.6875,5198945.5,1379.442,27.55271,-0.872281,-0.489005,60.7248,1,35.023


### GWR Binomial model with independent variable, x = slope

In [5]:
coords = list(zip(data_p['X'],data_p['Y']))
y = np.array(data_p['Landslid']).reshape((-1,1)) 
elev = np.array(data_p['Elev']).reshape((-1,1))
slope = np.array(data_p['Slope']).reshape((-1,1))
SinAspct = np.array(data_p['SinAspct']).reshape(-1,1)
CosAspct = np.array(data_p['CosAspct']).reshape(-1,1)
X = np.hstack([elev,slope,SinAspct,CosAspct])
x = SinAspct

X_std = (X-X.mean(axis=0))/X.std(axis=0)
x_std = (x-x.mean(axis=0))/x.std(axis=0)
y_std = (y-y.mean(axis=0))/y.std(axis=0)

In [16]:
bw=Sel_BW(coords,y,x_std,family=Binomial(),constant=False).search()

In [17]:
bw

100.0

In [21]:
gwr_mod=GWR(coords,y,x_std,bw=bw,family=Binomial(),constant=False).fit()

In [22]:
gwr_mod.aic

330.2296364641693

##### Running the function with family = Binomial()

In [23]:
selector = Sel_BW(coords,y,x_std,family=Binomial(),multi=True,constant=False)

In [24]:
selector.search(verbose=True)

Current iteration: 1 ,SOC: 0.1493438
Bandwidths: 82.0
Current iteration: 2 ,SOC: 0.0
Bandwidths: 82.0


array([82.])

In [25]:
mgwr_mod = MGWR(coords, y,x_std,selector,family=Binomial(),constant=False).fit()

In [26]:
mgwr_mod.aic

337.76149729129276

In [28]:
mgwr_mod.predy-gwr_mod.predy

array([[-1.88213249e-02],
       [ 2.96745385e-02],
       [ 2.19540320e-02],
       [ 3.53172641e-02],
       [ 3.34979652e-02],
       [-2.30589775e-02],
       [-4.13226775e-02],
       [-2.72275526e-02],
       [-2.32937485e-02],
       [-3.19027042e-02],
       [-4.10487815e-02],
       [-1.67073170e-02],
       [-3.33779404e-02],
       [ 2.31517027e-02],
       [ 2.69000788e-02],
       [ 4.78106415e-02],
       [-1.08443839e-02],
       [-1.91627481e-02],
       [ 1.13068279e-02],
       [ 5.33160983e-02],
       [-1.97387817e-03],
       [-4.25318200e-03],
       [-2.98913370e-02],
       [ 2.61567304e-02],
       [ 1.67728765e-02],
       [ 1.03608457e-02],
       [-1.56804715e-02],
       [ 1.09515972e-02],
       [ 1.44240532e-02],
       [-5.06887179e-03],
       [-1.41329638e-02],
       [ 2.61622645e-02],
       [-4.85361576e-02],
       [ 3.19017261e-02],
       [ 2.50631467e-02],
       [-9.35863419e-05],
       [ 3.10685026e-03],
       [ 4.08331844e-03],
       [ 1.0